In [121]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [122]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\user\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [123]:
def extract_data_points(job,div):
    for a in div.findAll('a',attrs={'class':'jobtitle turnstileLink'}):
        job['Title']=a['title']
    for a1 in div.findAll('a',attrs={'data-tn-element':'companyName'}):
        job['CompanyName']=a1.text.strip()
    for span in div.findAll('span',attrs={'class':'ratingsContent'}):
        job['Rating']=span.text.strip()
    for span1 in div.findAll('span',attrs={'class':'location accessible-contrast-color-location'}):
        job['Location']=span1.text.strip()
    for div1 in div.findAll('div',attrs={'class':'summary'}):
        summary=div1.text.strip()
        job['Summary']=re.sub('  +',' ',summary.replace("\n",""))
    for span2 in div.findAll('span',attrs={'class':'date'}):
        job['date']=span2.text.strip()
    return job

In [124]:
def get_data_from_webpage(data_collected,soup):
    job_posts=[]
    for div in soup.findAll('div',attrs={'class':'row'}):
        job=dict()
        job=extract_data_points(job,div)
        job_posts.append(div['data-jk'])
        single_job_post_extension_url="https://www.indeed.com/viewjob?jk="+div['data-jk']
        job["Url"]=single_job_post_extension_url
        driver.get(single_job_post_extension_url)
        job_soup=BeautifulSoup(driver.page_source,"html.parser")
        for inside_div in job_soup.findAll('div',attrs={'class':'jobsearch-jobDescriptionText'}):
            details=inside_div.text.strip()[:100]+"..."
            job["Details"]=re.sub(' +',' ',details.replace("\n"," "))
        data_collected.append(job)
    return data_collected
            

In [125]:
def scrape_data(url_to_scrape,number_of_pages_nos):
    data_collected=[]
    for i in range(0,number_of_pages_nos):
        extension=""
        if i is not 0:
            extension="&start="+str(i*10)
            url=url_to_scrape+extension
            driver.get(url)
            soup=BeautifulSoup(driver.page_source,"html.parser")
            data_collected=get_data_from_webpage(data_collected,soup)
    return data_collected

In [126]:
BASE_URL= "https://in.indeed.com/"
name_of_city="Banglore"
keywords="Data Scientist"
number_of_pages=10
name_of_city='+'.join(name_of_city.split(' '))
keywords='+'.join(keywords.split(' '))
url_to_scrape=BASE_URL +"/jobs?q="+keywords+"&l="+name_of_city
number_of_pages_nos=int(number_of_pages)
data_collected=scrape_data(url_to_scrape,number_of_pages_nos)

In [127]:
data_collected

[{'Title': 'Data Scientist',
  'CompanyName': 'Daimler',
  'Rating': '4.1',
  'Location': 'Bengaluru, Karnataka',
  'Summary': 'Good knowledge of statistical data analysis.Data processing and basic statistical analysis.Must hands-on experience in dashboard building using Power _BI and…',
  'date': '17 days ago',
  'Url': 'https://www.indeed.com/viewjob?jk=6f105d370ba703bf',
  'Details': 'Data Scientist Description Responsibilities: Data processing and basic statistical analysis Must han...'},
 {'Title': 'Data Scientist - Advanced Machine Learning',
  'Location': 'Bengaluru, Karnataka',
  'Summary': '>3 years’ experience in data science/Analytics roles.Develop and maintain robust data processing pipelines and reproducible modelling pipelines.',
  'date': '30+ days ago',
  'Url': 'https://www.indeed.com/viewjob?jk=3a928fe060caf5b4',
  'Details': 'Role Summary We are looking for Data Scientists to help us create innovative machine learning solut...'},
 {'Title': 'Data Scientist - Machine 

In [128]:
data=pd.DataFrame(data_collected)
data.head()

,Title,CompanyName,Rating,Location,Summary,date,Url,Details
0,Data Scientist,Daimler,4.1,"Bengaluru, Karnataka",Good knowledge of statistical data analysis.Da...,17 days ago,https://www.indeed.com/viewjob?jk=6f105d370ba7...,Data Scientist Description Responsibilities: D...
1,Data Scientist - Advanced Machine Learning,NaN,NaN,"Bengaluru, Karnataka",>3 years’ experience in data science/Analytics...,30+ days ago,https://www.indeed.com/viewjob?jk=3a928fe060ca...,Role Summary We are looking for Data Scientist...
2,"Data Scientist - Machine Learning, Deep Learni...",Enterprise Bot,3.3,"Bengaluru, Karnataka",The data scientist will be responsible for des...,3 days ago,https://www.indeed.com/viewjob?jk=505a185a491d...,"At Enterprise Bot, we create AI-powered bots t..."
3,DATA SCIENTIST,NaN,NaN,"Bengaluru, Karnataka",Working experience in end to end data science ...,8 days ago,https://www.indeed.com/viewjob?jk=07fcb11ddcfe...,We have an urgent requirement for Data Scienti...
4,Data Scientist,NaN,NaN,"Bengaluru, Karnataka",Build models and analyze medical data collecte...,30+ days ago,https://www.indeed.com/viewjob?jk=20213db8f043...,LOCATION Bangalore TYPE Full Time Looking for:...


In [129]:
data.to_csv("jobs.csv")